# 1.Sparkify 项目 Workspace
这个 Workspace 包括一个迷你的子数据集（128MB），是完整数据集（12GB）的一个子集。在将你的项目部署到云上之前，你可以自由使用 Workspace 来创建你的项目或用Spark来探索这个较小数据集。设置 Spark 集群的指南可以在选修 Spark 课程的内容里找到。

你可以依照下面的步骤进行项目的数据分析和模型搭建部分。

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

# import spark lib
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

# 导入datatime库，便于查看运行时间
import datetime
from time import time

> 由于项目要求，需要提供github项目地址，但github免费版对上传数据有限制，不能上传大于100MB的文件，需要将原有128MB数据集进行压缩，压缩格式使用bz2，在此格式下，pyspark可以直接读取数据集。

- Mac OS下，通过Terminal使用如下命令将数据集压缩

```
$ bzip2 -z mini_sparkify_event_data.json
```

压缩后数据容量大幅减小，由原来的128MB变为6.4MB

> **注意：使用此命令后会删除被压缩的文件**

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify_Spark_Sql_Session") \
    .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.driver.port', '36335'),
 ('spark.driver.host', '7fcd49c4889b'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'Sparkify_Spark_Sql_Session'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1581943848479')]

# 2.加载和清洗数据
在这个 Workspace 中，小数据集的名称是 `mini_sparkify_event_data.json.bz2`.加载和清洗数据集，检查是否有无效或缺失数据——例如，没有userid或sessionid的数据。 

## 2.1-数据的加载与观察

In [4]:
# 加载数据集
df = spark.read.json('mini_sparkify_event_data.json.bz2')

In [7]:
# 查看数据集描述（方法1）
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
# 查看数据集描述（方法2）
df.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [5]:
# 查看第一行数据
df.show(n=1)

+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|       location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
only showi

In [6]:
# 查看前5行数据
df.take(5)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [20]:
# 查看部分含义不明确的变量
df.describe('auth', 'itemInSession', 'length', 'method', 'page', 'ts').show()

+-------+----------+------------------+-----------------+------+-------+--------------------+
|summary|      auth|     itemInSession|           length|method|   page|                  ts|
+-------+----------+------------------+-----------------+------+-------+--------------------+
|  count|    286500|            286500|           228108|286500| 286500|              286500|
|   mean|      null|114.41421291448516|249.1171819778458|  null|   null|1.540956889810483...|
| stddev|      null|129.76726201140994|99.23517921058361|  null|   null|1.5075439608226302E9|
|    min| Cancelled|                 0|          0.78322|   GET|  About|       1538352117000|
|    max|Logged Out|              1321|       3024.66567|   PUT|Upgrade|       1543799476000|
+-------+----------+------------------+-----------------+------+-------+--------------------+



In [21]:
df.describe('userAgent').show()

+-------+--------------------+
|summary|           userAgent|
+-------+--------------------+
|  count|              278154|
|   mean|                null|
| stddev|                null|
|    min|"Mozilla/5.0 (Mac...|
|    max|Mozilla/5.0 (comp...|
+-------+--------------------+



**其中auth、method、page、userAgent为分类变量，适合进一步了解其含义。**

In [27]:
df.select('auth').dropDuplicates().sort('auth').show()

+----------+
|      auth|
+----------+
| Cancelled|
|     Guest|
| Logged In|
|Logged Out|
+----------+



In [28]:
df.select('method').dropDuplicates().sort('method').show()

+------+
|method|
+------+
|   GET|
|   PUT|
+------+



In [26]:
df.select('page').dropDuplicates().sort('page').show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



In [29]:
df.select('userAgent').dropDuplicates().sort('userAgent').show()

+--------------------+
|           userAgent|
+--------------------+
|                null|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Win...|
|"Mozilla/5.0 (Win...|
+--------------------+
only showing top 20 rows



In [22]:
# 查看有多少行数据
df.count()

286500

**对变量的理解**  
artist: 艺术家  
auth: 登录状态  
firstName: 名  
gender: 性别  
itemInSession:(具体含义暂不明确)  
lastName: 姓氏  
length: 长度(具体含义暂不明确)  
level: 会员等级(包含付费会员与免费会员)  
location: 地区  
method: 方法(可能与page关联较大)  
page: 请求页面  
registration: 注册码
sessionId: 页面会话ID  
song: 歌曲
status: 状态  
ts: (具体含义暂不明确)  
userAgent: 用户代理(可能含义为用户所用浏览器类型，大部分为火狐与谷歌浏览器)  
userId: 用户ID

## 2.2-数据清理

In [ ]:
# 删除重复值


# 探索性数据分析
当你使用完整数据集时，通过加载小数据集，在 Spark 中完成基础操作来实现探索性数据分析。在这个 Workspace 中，我们已经提供给你一个你可以探索的小数据集。

### 定义客户流失

在你完成初步分析之后，创建一列 `Churn` 作为模型的标签。我建议你使用 `Cancellation Confirmation` 事件来定义客户流失，该事件在付费或免费客户身上都有发生。作为一个奖励任务，你也可以深入了解 `Downgrade` 事件。

### 探索数据
你定义好客户流失后，就可以执行一些探索性数据分析，观察留存用户和流失用户的行为。你可以首先把这两类用户的数据聚合到一起，观察固定时间内某个特定动作出现的次数或者播放音乐的数量。

# 特征工程
熟悉了数据之后，就可以构建你认为会对训练模型帮助最大的特征。要处理完整数据集，你可以按照下述步骤：
- 写一个脚本来从小数据集中提取你需要的特征
- 确保你的脚本可以拓展到大数据集上，使用之前教过的最佳实践原则
- 在完整数据集上运行你的脚本，按运行情况调试代码

如果是在教室的 workspace，你可以直接用里面提供的小数据集来提取特征。确保当你开始使用 Spark 集群的时候，把上述的成果迁移到大数据集上。

# 建模
将完整数据集分成训练集、测试集和验证集。测试几种你学过的机器学习方法。评价不同机器学习方法的准确率，根据情况调节参数。根据准确率你挑选出表现最好的那个模型，然后报告在训练集上的结果。因为流失顾客数据集很小，我建议选用 F1 score 作为优化指标。

# 最后一步
清理你的代码，添加注释和重命名变量，使得代码更易读和易于维护。参考 Spark 项目概述页面和数据科学家毕业项目审阅要求，确保你的项目包含了毕业项目要求的所有内容，并且满足所有审阅要求。记得在 GitHub 代码库里包含一份全面的文档——README文件，以及一个网络应用程序或博客文章。